<a href="https://colab.research.google.com/github/NathanyApSalles/analysis_foodtech/blob/main/Case_Tecnico_DataAnalyst_Ifood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bases

In [2]:
import requests
import tarfile
import os
from pyspark.sql import SparkSession, DataFrame

spark = SparkSession.builder.getOrCreate()

In [3]:
def read_file(url: str, local_path: str, type_file: str) -> DataFrame:
  """Função para ler arquivo e retornar um Dataframe."""
  if not os.path.exists(local_path):
    response = requests.get(url)
    with open(local_path, "wb") as f:
        f.write(response.content)
  if type_file == "json":
    return spark.read.json(local_path, multiLine=False)
  elif type_file == "csv":
    return spark.read.option("header", "true").csv(local_path)
  elif type_file == "tar":

    arquivos_extraidos = "/tmp/ab_test_ref"

    os.makedirs(arquivos_extraidos, exist_ok=True)

    with tarfile.open(local_path, "r:gz") as tar:
        tar.extractall(path=arquivos_extraidos)

    filename = ""
    for root, dirs, files in os.walk(arquivos_extraidos):
        for filename in files:
            print(filename) # print para visualizar todos os arquivos exraídos
    if ".csv" in filename: # se houver algum arquivo csv, junte todos os arquivos deste tipo no dataframe
      return spark.read.option("header", "true").csv(arquivos_extraidos + "/*.csv")
    else: # se houver algum arquivo json, junte todos os arquivos deste tipo no dataframe
      return spark.read.json(arquivos_extraidos + "/*.json")
      #pode acontecer de ter arquivos de diferentes tipos misturados, mas para este estudo vamos assumir que todos são do mesmo tipo


In [4]:
url_pedidos = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz"
url_usuarios = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz"
url_merchants = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz"
url_test_ab = "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"

local_path_pedidos = "/tmp/order.json.gz"
local_path_usuarios = "/tmp/consumer.csv.gz"
local_path_merchants = "/tmp/restaurant.csv.gz"
local_path_teste_ab = "/tmp/ab_test_ref.tar.gz"

pedidos = read_file(url_pedidos, local_path_pedidos, "json").cache()
usuarios = read_file(url_usuarios, local_path_usuarios, "csv").cache()
merchants = read_file(url_merchants, local_path_merchants, "csv").cache()
teste_ab = read_file(url_test_ab, local_path_teste_ab, "tar").cache()

._ab_test_ref.csv
ab_test_ref.csv


In [4]:
pedidos.show(5)

+-----------+--------------------+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+--------------------+--------------------+-----------------+------------------+-----------------+--------------------+--------------------+---------------+--------------------+------------------+---------------+
|        cpf|         customer_id|customer_name|delivery_address_city|delivery_address_country|delivery_address_district|delivery_address_external_id|delivery_address_latitude|delivery_address_longitude|delivery_address_state|delivery_address_zip_code|               items|         merchant_id|merchant_latitude|merchant_longitude|merchant_timezone|    order_created_at|            order_id|order_scheduled|order_scheduled_date|order_total_amount|origin_platform|
+-----------+--------------------+-------------+---------------------+--

In [5]:
usuarios.show(5)

+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|         customer_id|language|          created_at|active|customer_name|customer_phone_area|customer_phone_number|
+--------------------+--------+--------------------+------+-------------+-------------------+---------------------+
|e8cc60860e09c0bb1...|   pt-br|2018-04-05T14:49:...|  true|         NUNO|                 46|            816135924|
|a2834a38a9876cf74...|   pt-br|2018-01-14T21:40:...|  true|     ADRIELLY|                 59|            231330577|
|41e1051728eba1334...|   pt-br|2018-01-07T03:47:...|  true|        PAULA|                 62|            347597883|
|8e7c1dcb64edf95c9...|   pt-br|2018-01-10T22:17:...|  true|       HELTON|                 13|            719366842|
|7823d4cf4150c5dae...|   pt-br|2018-04-06T00:16:...|  true|       WENDER|                 76|            543232158|
+--------------------+--------+--------------------+------+-------------

In [6]:
merchants.show(5)

+--------------------+--------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|                  id|          created_at|enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code| merchant_city|merchant_state|merchant_country|
+--------------------+--------------------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+--------------+--------------+----------------+
|d19ff6fca6288939b...|2017-01-23T12:52:...|  false|          3|          60.0|           0|           50|               30.0|            14025|RIBEIRAO PRETO|            SP|              BR|
|631df0985fdbbaf27...|2017-01-20T13:14:...|   true|          3|          60.0|           0|            0|               30.0|            50180|     SAO PAULO|            SP|              BR|
|135c5c4ae4c1ec1fd...|2017-01-23T12:46:...|  

In [7]:
teste_ab.show(5, truncate=False)

+----------------------------------------------------------------+---------+
|customer_id                                                     |is_target|
+----------------------------------------------------------------+---------+
|755e1fa18f25caec5edffb188b13fd844b2af8cf5adedcf77c028f36cb9382ea|target   |
|b821aa8372b8e5b82cdc283742757df8c45eecdd72adf411716e710525d4edf1|control  |
|d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc007f5297541ec40b63b|control  |
|6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c8b535718bbc651ab02|target   |
|dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f5e3763fa657c23a58d|control  |
+----------------------------------------------------------------+---------+
only showing top 5 rows



In [5]:
from pyspark.sql.functions import col, broadcast, when, count, isnan, count_distinct, sum, avg, row_number
from pyspark.sql.window import Window

# Entendendo os dados

In [6]:
def df_info(df: DataFrame, colunas: list) -> None:
  """Função para auxiliar a identificar tamanho da base, tipo das colunas,
   valores nulos, duplicidade em colunas específicas na base."""
  # validando o tipo das colunas
  df.printSchema()

  # validando o tamanho da base
  print(f"Qtd de linhas: {df.count()}")

  print("\nValores nulos")
  df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

  print("\nValidando valores duplicados")
  for column in colunas:
    num_duplicados = df.groupBy(column).count().where(col("count") > 1).count()
    print(f"\n{column}: {num_duplicados}")

In [10]:
df_info(usuarios, ["customer_id"])

root
 |-- customer_id: string (nullable = true)
 |-- language: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- active: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_phone_area: string (nullable = true)
 |-- customer_phone_number: string (nullable = true)

Qtd de linhas: 806156

Valores nulos
+-----------+--------+----------+------+-------------+-------------------+---------------------+
|customer_id|language|created_at|active|customer_name|customer_phone_area|customer_phone_number|
+-----------+--------+----------+------+-------------+-------------------+---------------------+
|          0|       0|         0|     0|            0|                  0|                    0|
+-----------+--------+----------+------+-------------+-------------------+---------------------+


Validando valores duplicados

customer_id: 0


In [11]:
df_info(pedidos, ["order_id"])

root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_scheduled_date: string (nullable = true)
 |--

In [7]:
# removendo os usuários nulos da base
# na base de pedidos temos um número pequeno de pedidos sem customer_id atribuído,
# porém como o objetivo da análise é validar os resultados do teste a/b, estes pedidos serão removidos visto que
# não participaram do teste a/b
pedidos_validos_ab = pedidos.where(col("customer_id").isNotNull())

In [8]:
# podemos ver que a base de pedidos possui uma grande voliumentria de pedidos duplicados
# foi analisado uma amostra e notou-se que para os pedidos duplicados, os campos que diferente são CPF e data de criação do pedido
# pode ter acontecido algum problema no produto ao gerar o número do pedido, ou até mesmo ao gerar a base,
# diantes disto podemos seguir com algumas tratativas, como criar um novo order_id concatenando com o CPF, ou
# dentro dos duplicados manter o pedido mais atual, ou a mais antigo.
# Como os valores dos pedidos são iguais, merchants, itens também são idênticos, optou-se por manter o pedido mais antigo


pedidos_validos_ab = (pedidos_validos_ab
                      .withColumn("rank", row_number().over(Window.partitionBy("order_id").orderBy("order_created_at")))
                      .where(col("rank") == 1)
                      .drop("rank")
).cache()
df_info(pedidos_validos_ab, ["order_id"])

root
 |-- cpf: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delivery_address_city: string (nullable = true)
 |-- delivery_address_country: string (nullable = true)
 |-- delivery_address_district: string (nullable = true)
 |-- delivery_address_external_id: string (nullable = true)
 |-- delivery_address_latitude: string (nullable = true)
 |-- delivery_address_longitude: string (nullable = true)
 |-- delivery_address_state: string (nullable = true)
 |-- delivery_address_zip_code: string (nullable = true)
 |-- items: string (nullable = true)
 |-- merchant_id: string (nullable = true)
 |-- merchant_latitude: string (nullable = true)
 |-- merchant_longitude: string (nullable = true)
 |-- merchant_timezone: string (nullable = true)
 |-- order_created_at: string (nullable = true)
 |-- order_id: string (nullable = true)
 |-- order_scheduled: boolean (nullable = true)
 |-- order_scheduled_date: string (nullable = true)
 |--

In [14]:
df_info(merchants, ["id"])

root
 |-- id: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- enabled: string (nullable = true)
 |-- price_range: string (nullable = true)
 |-- average_ticket: string (nullable = true)
 |-- takeout_time: string (nullable = true)
 |-- delivery_time: string (nullable = true)
 |-- minimum_order_value: string (nullable = true)
 |-- merchant_zip_code: string (nullable = true)
 |-- merchant_city: string (nullable = true)
 |-- merchant_state: string (nullable = true)
 |-- merchant_country: string (nullable = true)

Qtd de linhas: 7292

Valores nulos
+---+----------+-------+-----------+--------------+------------+-------------+-------------------+-----------------+-------------+--------------+----------------+
| id|created_at|enabled|price_range|average_ticket|takeout_time|delivery_time|minimum_order_value|merchant_zip_code|merchant_city|merchant_state|merchant_country|
+---+----------+-------+-----------+--------------+------------+-------------+-------------------+---

In [15]:
df_info(teste_ab, ["customer_id"])

root
 |-- customer_id: string (nullable = true)
 |-- is_target: string (nullable = true)

Qtd de linhas: 806467

Valores nulos
+-----------+---------+
|customer_id|is_target|
+-----------+---------+
|          0|        0|
+-----------+---------+


Validando valores duplicados

customer_id: 0


# EDA

In [13]:
# tem no teste_Ab mas não comprou? ou seja, não está na base de pedidos?
c = pedidos_validos_ab.select(col("customer_id").alias("custumer_with_order")).distinct()
teste_ab.join(c, col("customer_id") == col("custumer_with_order"), "left").where(col("custumer_with_order").isNull()).show()

+-----------+---------+-------------------+
|customer_id|is_target|custumer_with_order|
+-----------+---------+-------------------+
|       null|   target|               NULL|
+-----------+---------+-------------------+



In [ ]:
# todos os usuários do teste a/b tem algum pedido, logo,
# uma sugestão é ter uma base sobre todas os usuários que receberam cupons para conseguirmos calcular conversão

In [14]:
teste_ab.groupBy(col("is_target")).agg(count_distinct("customer_id")).show()

+---------+---------------------------+
|is_target|count(DISTINCT customer_id)|
+---------+---------------------------+
|  control|                     360542|
|   target|                     445925|
+---------+---------------------------+



In [15]:
pedidos_final = pedidos_validos_ab.join(broadcast(teste_ab), ['customer_id'], "left").cache()
#pedidos_final.show(5)

In [16]:
pedidos_final.where(col("is_target").isNull()).show()

+-----------+---+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+--------------------------+----------------------+-------------------------+-----+-----------+-----------------+------------------+-----------------+----------------+--------+---------------+--------------------+------------------+---------------+---------+
|customer_id|cpf|customer_name|delivery_address_city|delivery_address_country|delivery_address_district|delivery_address_external_id|delivery_address_latitude|delivery_address_longitude|delivery_address_state|delivery_address_zip_code|items|merchant_id|merchant_latitude|merchant_longitude|merchant_timezone|order_created_at|order_id|order_scheduled|order_scheduled_date|order_total_amount|origin_platform|is_target|
+-----------+---+-------------+---------------------+------------------------+-------------------------+----------------------------+-------------------------+-------

In [35]:
pedidos_final.groupBy(col("is_target")).agg(
                                      count_distinct(col("order_id")).alias("distinct_orders"),
                                      count_distinct(col("customer_id")).alias("distinct_customers"),
                                      count_distinct(col("merchant_id")).alias("distinct_merchants"),
                                      sum(col("order_total_amount")).alias("order_total_amount"),
                                      avg(col("order_total_amount")).alias("order_avg_amount"),
                                      (sum(col("order_total_amount"))/count_distinct(col("order_id"))).alias("ticket_medio"),


                                      ).show()

+---------+---------------+------------------+------------------+-------------------+-----------------+-----------------+
|is_target|distinct_orders|distinct_customers|distinct_merchants| order_total_amount| order_avg_amount|     ticket_medio|
+---------+---------------+------------------+------------------+-------------------+-----------------+-----------------+
|  control|        1010738|            360542|              7196|4.843220348999915E7|47.91766361806833|47.91766361806833|
|   target|        1416677|            445924|              7227| 6.77299864499999E7|47.80905347513929|47.80905347513929|
+---------+---------------+------------------+------------------+-------------------+-----------------+-----------------+



In [ ]:
tempo_entre_compras =



---


# Teste de Hipótese:

In [22]:
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import shapiro, anderson, boxcox, mannwhitneyu
import numpy as np
import seaborn as sns

In [18]:
#qtd de pedidos por usuário
df_media_pedidos = pedidos_final.groupBy(col("customer_id"), col("is_target")).agg(count_distinct("order_id").alias("qtd_pedidos_por_usuario"))

df_media_pedidos.groupBy(col("is_target")).agg(avg(col("qtd_pedidos_por_usuario")).alias("media_pedidos_por_usuario")).show()

+---------+-------------------------+
|is_target|media_pedidos_por_usuario|
+---------+-------------------------+
|  control|         2.80338490383922|
|   target|        3.176947192795185|
+---------+-------------------------+



# Usuários target compram mais do que usuários control?
Primeiramente vamos validar se os dados possuem uma distribuição normal

##Teste de Shapiro Wilk

**- H0:** A amostra segue uma distribuição normal

**- H1:** A amostra não segue uma distribuição normal
- Se _p-value_ < 0.05, rejeita-se a hipótese nula, ou seja, temos evidências suficientes para dizer que a amostra não vem de distribuição normal


In [23]:
pd_pedidos = df_media_pedidos.toPandas()

In [24]:
# analisando se os dados obedecem uma distribuição normal
statistic, pvalue = shapiro(pd_pedidos["qtd_pedidos_por_usuario"])
print(f"statistic={statistic}, pvalue={pvalue}")
if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

statistic=0.5633321475085724, pvalue=8.558000673904252e-203
Rejeita a hipótese nula


/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:586: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 806466.
  res = hypotest_fun_out(*samples, **kwds)


Como os dados não seguem uma distribuição normal, logo, seguiremos por testes não paramétricos

## Hipóteses
**H0:** quantidade de pedidos do grupo target é igual ao grupo control

**H1:** quantidade de pedidos do grupo target é diferente ao grupo control

In [33]:
target = pd_pedidos[pd_pedidos["is_target"]=="target"]
control = pd_pedidos[pd_pedidos["is_target"]=="control"]

statistic, pvalue = mannwhitneyu(target["qtd_pedidos_por_usuario"], control["qtd_pedidos_por_usuario"])
print(f"statistic={statistic}, pvalue={pvalue}")

if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

#tamanho de efeito

print(statistic/(len(target["qtd_pedidos_por_usuario"])*len(control["qtd_pedidos_por_usuario"])))

statistic=88361946996.0, pvalue=0.0
Rejeita a hipótese nula
0.5496023311179173


## Resultado:
Rejeitamos a hipótese nula (H₀) e aceitamos a hipótese alternativa, ou seja, há uma diferença significativa entre a quantidade de pedidos dos dois grupos. O tamanho de efeito nos informa a magnitude desta diferença, sendo um índice entre 0 e 1. O tamanho do efeito calculado é 0.55, o que sugere um efeito de tamanho moderado. Isso significa que a diferença na quantidade de pedidos é significativa além de ser relevante.


---


## Vamos analisar agora se a diferença é para mais ou para menos
**H0:** usuários do grupo target tem igual ou menor quatidade de pedidos que o contro control

**H1:** usuários do grupo target tem maior quatidade de pedidos que o contro control

In [34]:

target = pd_pedidos[pd_pedidos["is_target"]=="target"]
control = pd_pedidos[pd_pedidos["is_target"]=="control"]

statistic, pvalue = mannwhitneyu(target["qtd_pedidos_por_usuario"], control["qtd_pedidos_por_usuario"], alternative='greater')
print(f"statistic={statistic}, pvalue={pvalue}")

if pvalue < 0.05:
    print('Rejeita a hipótese nula')
else:
    print('Não rejeita a hipótese nula')

#tamanho de efeito

print(statistic/(len(target["qtd_pedidos_por_usuario"])*len(control["qtd_pedidos_por_usuario"])))

statistic=88361946996.0, pvalue=0.0
Rejeita a hipótese nula
0.5496023311179173


## Resultado do teste de hipótese:
Novamente rejeitamos a hipótese nula (H₀) concluindo que o grupo target realizou mais pedidos em relação ao grupo control
O tamanho de efeito nos informa a magnitude desta diferença, sendo um índice entre 0 e 1. O tamanho do efeito calculado é 0.55, o que sugere um efeito de tamanho moderado. Isso significa que a diferença no tempo de atendimento é significativa além de ser relevante.



---




- tarquet x control

    - qtd de pedidos
    - valor total da compra
    - ticket médio
    - tempo entre compras
    - recorrencia
    - tem diferença entre usuários ativos ou não?
    - expansão geográfica: qtd de estabelecimentos diferentes que compraram
    - restaurantes que mais venderam
    - região que mais vendeu
    - horário das compras
    - qtd de produtos adquiridos

- ideias para teste ab:

    - qtd de usuários que receberam o cupom, para entender conversão, de quem recebeu, quem não comprou;
     